In [1]:
from vivarium import Artifact
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from db_queries import get_ids, get_outputs
import scipy.stats

!whoami
!date

alibow
Tue Jul  7 14:42:10 PDT 2020


# NOTE: Conclusion statements in this notebook have not been updated

In [2]:
output_dirs = ['/share/costeffectiveness/results/vivarium_conic_lsff/nigeria/2020_06_26_20_28_27/count_data/']

locations = ['Nigeria']

# 1. Iron effect on birth weight

In [3]:
births = pd.read_hdf(output_dirs[0] + 'births.hdf')
births.head()

# note: no stratification of birth counts by iron coverage group

,year,sex,folic_acid_fortification_group,measure,input_draw,scenario,value
0,2020,female,covered,live_births,21,baseline,3325.0
1,2020,female,covered,live_births,21,baseline,5.0
2,2020,female,covered,live_births,21,baseline,249.0
3,2020,female,covered,live_births,21,folic_acid_fortification_scale_up,3325.0
4,2020,female,covered,live_births,21,folic_acid_fortification_scale_up,5.0


In [6]:
bw = pd.read_hdf(output_dirs[0] + 'birth_weight.hdf')
bw = bw.loc[bw.year == '2023']
bw.head(10)

,year,sex,measure,input_draw,scenario,value,iron_fortification_group
2400,2023,female,birth_weight_mean,21,baseline,3194.152782,uncovered
2401,2023,female,birth_weight_mean,21,baseline,3199.771192,covered
2402,2023,female,birth_weight_mean,21,folic_acid_fortification_scale_up,3194.152782,uncovered
2403,2023,female,birth_weight_mean,21,folic_acid_fortification_scale_up,3199.771192,covered
2404,2023,female,birth_weight_mean,21,iron_folic_acid_fortification_scale_up,3192.312105,uncovered
2405,2023,female,birth_weight_mean,21,iron_folic_acid_fortification_scale_up,3201.689917,covered
2406,2023,female,birth_weight_mean,21,vitamin_a_fortification_scale_up,3194.152782,uncovered
2407,2023,female,birth_weight_mean,21,vitamin_a_fortification_scale_up,3199.771192,covered
2408,2023,female,birth_weight_mean,29,baseline,3095.355615,uncovered
2409,2023,female,birth_weight_mean,29,baseline,3104.129630,covered


In [5]:
bw_by_coverage_and_scenario = bw.loc[bw.measure == 'birth_weight_mean']
bw_by_coverage_and_scenario = bw_by_coverage_and_scenario.groupby(['scenario','iron_fortification_group']).describe()
bw_by_coverage_and_scenario['value']

# we would expect mean birth weight to be higher in covered group than uncovered group (for all scenarios)
# we are seeing the opposite
# but Kjell verified that the MEDIAN birth weight in covered group > median birth weight in uncovered group
    # this suggests that the distribution is differentially skewed high
    
# we would also expect to see simular mean birth weights by coverage group between scenarios
    # we are not seeing this.

count  \
scenario                               iron_fortification_group          
baseline                               covered                    50.0   
                                       uncovered                  50.0   
folic_acid_fortification_scale_up      covered                    50.0   
                                       uncovered                  50.0   
iron_folic_acid_fortification_scale_up covered                    50.0   
                                       uncovered                  50.0   
vitamin_a_fortification_scale_up       covered                    50.0   
                                       uncovered                  50.0   

                                                                        mean  \
scenario                               iron_fortification_group                
baseline                               covered                   3186.654842   
                                       uncovered                 3185.132276   
folic_acid_fortification_scale_up      covered                   3186.654842   
                                       uncovered                 3185.132276   
iron_folic_acid_fortification_scale_up covered                   3185.208929   
                                       uncovered                 3185.273724   
vitamin_a_fortification_scale_up       covered                   3186.654842   
                                       uncovered                 3185.132276   

                                                                       std  \
scenario                               iron_fortification_group              
baseline                               covered                   70.800133   
                                       uncovered                 71.213502   
folic_acid_fortification_scale_up      covered                   70.800133   
                                       uncovered                 71.213502   
iron_folic_acid_fortification_scale_up covered                   72.063591   
                                       uncovered                 70.873879   
vitamin_a_fortification_scale_up       covered                   70.800133   
                                       uncovered                 71.213502   

                                                                         min  \
scenario                               iron_fortification_group                
baseline                               covered                   3009.795830   
                                       uncovered                 3026.618758   
folic_acid_fortification_scale_up      covered                   3009.795830   
                                       uncovered                 3026.618758   
iron_folic_acid_fortification_scale_up covered                   3020.193458   
                                       uncovered                 3026.949706   
vitamin_a_fortification_scale_up       covered                   3009.795830   
                                       uncovered                 3026.618758   

                                                                         25%  \
scenario                               iron_fortification_group                
baseline                               covered                   3133.477888   
                                       uncovered                 3123.605752   
folic_acid_fortification_scale_up      covered                   3133.477888   
                                       uncovered                 3123.605752   
iron_folic_acid_fortification_scale_up covered                   3118.756958   
                                       uncovered                 3125.535541   
vitamin_a_fortification_scale_up       covered                   3133.477888   
                                       uncovered                 3123.605752   

                                                                         50%  \
scenario                               iron_fortification_gr

## Birth weight conclusions

- Cannot calculate overall mean birth weight by scenario because births are not stratified by iron coverage group
- Mean birthweight is lower in the covered group than the uncovered group... this is the opposite of what we would expect
    - Kjell verified that the MEDIAN birth weight is higher in the covered group than uncovered group, indicating that this is likely working as expected
- Mean birth weight for each covered group should be approximately equal between scenarios, although we are not seeing this behavior

**So, this is tentatively thought to be working as intended**

# 2. Vitamin A Fortification and Vitamin A Deficiency Prevalence

In [6]:
state_pt = pd.read_hdf(output_dirs[0] + 'state_person_time.hdf')
state_pt.head()

,year,age_group,cause,folic_acid_fortification_group,vitamin_a_fortification_group,measure,input_draw,scenario,value
0,2020,1_to_4,diarrheal_diseases,covered,covered,person_time,21,baseline,0.0
1,2020,1_to_4,diarrheal_diseases,covered,covered,person_time,21,folic_acid_fortification_scale_up,0.0
2,2020,1_to_4,diarrheal_diseases,covered,covered,person_time,21,iron_folic_acid_fortification_scale_up,0.0
3,2020,1_to_4,diarrheal_diseases,covered,covered,person_time,21,vitamin_a_fortification_scale_up,0.0
4,2020,1_to_4,diarrheal_diseases,covered,covered,person_time,29,baseline,0.0


In [7]:
def calculate_stratified_vad_prevalence(strata_cols):
    state_pt = pd.read_hdf(output_dirs[0] + 'state_person_time.hdf')
    vad = state_pt.loc[state_pt.cause == 'vitamin_a_deficiency']
    vad = vad.groupby((['scenario','input_draw'] + strata_cols)).sum()
    pt = pd.read_hdf(output_dirs[0] + 'state_person_time.hdf')
    pt = pt.loc[pt.cause.str.contains('diarrheal')]
    pt = pt.groupby((['scenario','input_draw'] + strata_cols)).sum()
    vad_prev = vad / pt
    vad_prev = vad_prev.reset_index()
    vad_prev = vad_prev.loc[vad_prev.scenario != 'iron_fortification_scale_up']
    vad_prev = (vad_prev.groupby((['scenario'] + strata_cols)).mean()
                .rename(columns={'value':'vad_prevalence'})
                .drop(columns='input_draw'))
    return vad_prev

In [8]:
overall = calculate_stratified_vad_prevalence([])
overall

# looks good! (Prevalence of VAD is lower in vitamin_a_fortification_scale_up scenario than baseline scenario)

,vad_prevalence
scenario,
baseline,0.227854
folic_acid_fortification_scale_up,0.227854
iron_folic_acid_fortification_scale_up,0.227854
vitamin_a_fortification_scale_up,0.209538


In [9]:
by_year = calculate_stratified_vad_prevalence(['year'])
by_year

# looks good! 
# VAD prevalence is the same in each scenario in 2020, and then lower in vit_a_scenaario in 2021-2025

vad_prevalence
scenario                               year                
baseline                               2020        0.228603
                                       2021        0.228014
                                       2022        0.227604
                                       2023        0.227219
folic_acid_fortification_scale_up      2020        0.228603
                                       2021        0.228014
                                       2022        0.227604
                                       2023        0.227219
iron_folic_acid_fortification_scale_up 2020        0.228603
                                       2021        0.228014
                                       2022        0.227604
                                       2023        0.227219
vitamin_a_fortification_scale_up       2020        0.228603
                                       2021        0.216165
                                       2022        0.200885
                                       2023        0.193184

In [10]:
by_age = calculate_stratified_vad_prevalence(['age_group'])
by_age

# no change in neonatal age groups between scenarios, as expected
# lower VAD prevalence in vitamin_a_scenario in post_neonatal and 1_to_4 ages, as expected
# looks good!

vad_prevalence
scenario                               age_group                     
baseline                               1_to_4                0.232407
                                       early_neonatal        0.228557
                                       late_neonatal         0.225594
                                       post_neonatal         0.209489
folic_acid_fortification_scale_up      1_to_4                0.232407
                                       early_neonatal        0.228557
                                       late_neonatal         0.225593
                                       post_neonatal         0.209489
iron_folic_acid_fortification_scale_up 1_to_4                0.232407
                                       early_neonatal        0.228557
                                       late_neonatal         0.225594
                                       post_neonatal         0.209489
vitamin_a_fortification_scale_up       1_to_4                0.211716
                                       early_neonatal        0.228557
                                       late_neonatal         0.225594
                                       post_neonatal         0.199266

In [11]:
by_coverage_group = calculate_stratified_vad_prevalence(['vitamin_a_fortification_group'])
by_coverage_group

# unexpected results here...

# we should expect...

    # all VAD_prev < 1 in each coverage group
    # covered VAD_prev ~= uncovered VAD_prev
    # effectively_covered VAD_prev * 2.22 ~= uncovered VAD_prev in both scenarios
    # all VAD_prev by coverage group in baseline scenario should ~= VAD_prev by coverage group in vitamin_a scenario

vad_prevalence
scenario                               vitamin_a_fortification_group                
baseline                               covered                              0.212355
                                       effectively_covered                  0.117905
                                       uncovered                            0.235598
folic_acid_fortification_scale_up      covered                              0.212356
                                       effectively_covered                  0.117905
                                       uncovered                            0.235598
iron_folic_acid_fortification_scale_up covered                              0.212357
                                       effectively_covered                  0.117905
                                       uncovered                            0.235598
vitamin_a_fortification_scale_up       covered                              0.226515
                                       effectively_covered                  0.118210
                                       uncovered                            0.235642

## Conclusions

- For the covered group, VAD state person time > overall person time, which appears to be an error (prevalence > 1)
- For the baseline scenario, VAD prevalence in the effectively covered group is ~ 0.45 times that in the uncovered group, as expected. However, in the vitamin A scenario this is not the case and the difference is greater between these groups -- we would expect to see the same rates here and just more person time in the covered groups in the vitamin A scenario.
- Otherwise, behavior by year, age group, and overall looks as expected
- VAD prevalence does not decrease until 2021, indicating that the coverage algorithm was implemented correctly

# 3. Iron effect on hemoglobin

In [12]:
hb = pd.read_hdf(output_dirs[0] + 'hemoglobin_level.hdf')
hb.head()

,sex,measure,input_draw,scenario,value,age,status,responsive
0,female,hemoglobin_mean,21,baseline,99.861305,0.5,covered,responsive
1,female,hemoglobin_mean,21,baseline,89.421314,0.5,covered,non-responsive
2,female,hemoglobin_mean,21,baseline,100.046889,0.5,uncovered,responsive
3,female,hemoglobin_mean,21,baseline,92.766277,0.5,uncovered,non-responsive
4,female,hemoglobin_mean,21,baseline,112.162315,1,covered,responsive


In [13]:
np.unique(hb['measure'])

array(['hemoglobin_mean', 'hemoglobin_variance'], dtype=object)

In [14]:
pt = pd.read_hdf(output_dirs[0] + 'person_time.hdf')
pt.head()

# NOTE: cannot calculate weighted averages of Hb because persontime data not stratified by iron coverage or responsiveness

,year,age_group,folic_acid_fortification_group,vitamin_a_fortification_group,measure,input_draw,scenario,value
0,2020,1_to_4,covered,covered,person_time,21,baseline,0.0
1,2020,1_to_4,covered,covered,person_time,21,folic_acid_fortification_scale_up,0.0
2,2020,1_to_4,covered,covered,person_time,21,iron_folic_acid_fortification_scale_up,0.0
3,2020,1_to_4,covered,covered,person_time,21,vitamin_a_fortification_scale_up,0.0
4,2020,1_to_4,covered,covered,person_time,29,baseline,0.0


In [15]:
def get_stratified_hb_mean(strata_cols):
    hb = pd.read_hdf(output_dirs[0] + 'hemoglobin_level.hdf')
    hb = hb.loc[hb.scenario != 'vitamin_a_fortification_scale_up']
    hb = hb.loc[hb.measure == 'hemoglobin_mean']
    hb = hb.groupby(['scenario','measure'] + strata_cols).mean()
    return hb

In [16]:
overall = get_stratified_hb_mean([])
overall


# looks good! (mean hemoglobin is slightly higher in iron_fortification scenario)

,,input_draw,value
scenario,measure,,
baseline,hemoglobin_mean,437.52,97.737758
folic_acid_fortification_scale_up,hemoglobin_mean,437.52,97.737754
iron_folic_acid_fortification_scale_up,hemoglobin_mean,437.52,98.191855


In [17]:
by_coverage = get_stratified_hb_mean(['status'])
by_coverage

# hemoglobin is lower in covered group, which is the opposite of what we would expect

# should expect...
    
    # covered mean hemoglobin to be higher than uncovered mean hemoglobin in both scenarios
        # note, if this is not true, it should be true for MEDIAN hemoglobin by coverage group
    # mean hemoglobin by coverage group should be approximately equal between scenarios
        # note, it is ok if covered hemoglobin in the iron scenario is slightly less than the covered 
        # group in the baseline scenario

input_draw  \
scenario                               measure         status                  
baseline                               hemoglobin_mean covered        437.52   
                                                       uncovered      437.52   
folic_acid_fortification_scale_up      hemoglobin_mean covered        437.52   
                                                       uncovered      437.52   
iron_folic_acid_fortification_scale_up hemoglobin_mean covered        437.52   
                                                       uncovered      437.52   

                                                                      value  
scenario                               measure         status                
baseline                               hemoglobin_mean covered    97.321805  
                                                       uncovered  98.153712  
folic_acid_fortification_scale_up      hemoglobin_mean covered    97.321805  
                                                       uncovered  98.153703  
iron_folic_acid_fortification_scale_up hemoglobin_mean covered    98.247857  
                                                       uncovered  98.135853

In [18]:
by_responsiveness = get_stratified_hb_mean(['status','responsive'])
by_responsiveness

# we are seeing some unexpected results here

    
# expected results:

    # for a given scenario...
        # covered, responsive hemoglobin > uncovered, responsive hemoglobin (by ~ 3 units)
        # covered, non-responsive hemoglobin ~= uncovered, non-responsive hemoglobin
    
    # mean hemoglobin by coverage group and responsiveness should be approximately equal between scenarios
        
    # for a given coverage group, responsive hemoglobin > non-responsive hemoglobin
        # for a given ANEMIA group (i.e. severe anemia)... responsive hemoglobin ~= non-responsive hemoglobin
            # can verify this in interactive sim?
        # note: difference between responsive and non-responsive should be larger in covered group than uncovered group

input_draw  \
scenario                               measure         status    responsive                   
baseline                               hemoglobin_mean covered   non-responsive      437.52   
                                                                 responsive          437.52   
                                                       uncovered non-responsive      437.52   
                                                                 responsive          437.52   
folic_acid_fortification_scale_up      hemoglobin_mean covered   non-responsive      437.52   
                                                                 responsive          437.52   
                                                       uncovered non-responsive      437.52   
                                                                 responsive          437.52   
iron_folic_acid_fortification_scale_up hemoglobin_mean covered   non-responsive      437.52   
                                                                 responsive          437.52   
                                                       uncovered non-responsive      437.52   
                                                                 responsive          437.52   

                                                                                      value  
scenario                               measure         status    responsive                  
baseline                               hemoglobin_mean covered   non-responsive   90.624500  
                                                                 responsive      104.019109  
                                                       uncovered non-responsive   93.632450  
                                                                 responsive      102.674975  
folic_acid_fortification_scale_up      hemoglobin_mean covered   non-responsive   90.624500  
                                                                 responsive      104.019109  
                                                       uncovered non-responsive   93.632452  
                                                                 responsive      102.674954  
iron_folic_acid_fortification_scale_up hemoglobin_mean covered   non-responsive   92.357642  
                                                                 responsive      104.138072  
                                                       uncovered non-responsive   93.598190  
                                                                 responsive      102.673516

In [19]:
by_age = get_stratified_hb_mean(['age'])
by_age

# would expect to see slight difference for 0.5 age group between scenarios
# expect to see larger differences between scenarios for other ages

input_draw  \
scenario                               measure         age               
baseline                               hemoglobin_mean 0.5      437.52   
                                                       1        437.52   
                                                       2        437.52   
                                                       3        437.52   
folic_acid_fortification_scale_up      hemoglobin_mean 0.5      437.52   
                                                       1        437.52   
                                                       2        437.52   
                                                       3        437.52   
iron_folic_acid_fortification_scale_up hemoglobin_mean 0.5      437.52   
                                                       1        437.52   
                                                       2        437.52   
                                                       3        437.52   

                                                                value  
scenario                               measure         age             
baseline                               hemoglobin_mean 0.5  94.961783  
                                                       1    98.429911  
                                                       2    98.796553  
                                                       3    98.762786  
folic_acid_fortification_scale_up      hemoglobin_mean 0.5  94.961771  
                                                       1    98.429905  
                                                       2    98.796553  
                                                       3    98.762786  
iron_folic_acid_fortification_scale_up hemoglobin_mean 0.5  95.461097  
                                                       1    98.884701  
                                                       2    99.221204  
                                                       3    99.200418

In [20]:
by_responsiveness = get_stratified_hb_mean(['responsive'])
by_responsiveness

# we would expect...

    # baseline non-responsive hemoglobin ~= iron scenario non-responsive hemoglobin
    # baseline responsive hemoglobin < iron scenario responsive hemoglobin (by < 3 units)
    
# we are seeing unexpected results here in that ...

    # baseline non-responsive hemoglobin < iron scenario non-responsive hemoglobin
    # baseline responsive hemoglobin = iron scenario responsive hemoglobin

input_draw  \
scenario                               measure         responsive                   
baseline                               hemoglobin_mean non-responsive      437.52   
                                                       responsive          437.52   
folic_acid_fortification_scale_up      hemoglobin_mean non-responsive      437.52   
                                                       responsive          437.52   
iron_folic_acid_fortification_scale_up hemoglobin_mean non-responsive      437.52   
                                                       responsive          437.52   

                                                                            value  
scenario                               measure         responsive                  
baseline                               hemoglobin_mean non-responsive   92.128475  
                                                       responsive      103.347042  
folic_acid_fortification_scale_up      hemoglobin_mean non-responsive   92.128476  
                                                       responsive      103.347031  
iron_folic_acid_fortification_scale_up hemoglobin_mean non-responsive   92.977916  
                                                       responsive      103.405794

## Conclusions

- hemoglobin is lower in covered group, which is the opposite of what we would expect
- there is significant variation in mean hemoglobin between responsive and non-responsive groups in the uncovered population, which is unexpected
- seeing a difference in mean hemoglobin between scenarios for the 0.5 age, which is unexpected
- seeing an increase in mean hemoglobin between scenarios in the non-responsive population, which is unexpected

# 4. Iron fortification on anemia prevalence

In [21]:
anemia_pt = pd.read_hdf(output_dirs[0] + 'anemia_state_person_time.hdf')
anemia_pt.head()

,year,age_group,sex,measure,input_draw,scenario,value
0,2020,1_to_4,female,anemia_mild,21,baseline,35566.425736
1,2020,1_to_4,female,anemia_mild,21,folic_acid_fortification_scale_up,35566.425736
2,2020,1_to_4,female,anemia_mild,21,iron_folic_acid_fortification_scale_up,35566.425736
3,2020,1_to_4,female,anemia_mild,21,vitamin_a_fortification_scale_up,35566.425736
4,2020,1_to_4,female,anemia_mild,29,baseline,41695.101985


In [22]:
def get_anemia_prevalence(strata_cols):
    anemia_pt = pd.read_hdf(output_dirs[0] + 'anemia_state_person_time.hdf')
    anemia_pt = anemia_pt.loc[anemia_pt.scenario != 'vitamin_a_fortification_scale_up']
    anemia_pt = anemia_pt.groupby((['scenario','input_draw'] + strata_cols + ['measure'])).sum()
    pt = pd.read_hdf(output_dirs[0] + 'state_person_time.hdf')
    pt = pt.loc[pt.cause.str.contains('diarrheal')]
    pt = pt.loc[pt.scenario != 'vitamin_a_fortification_scale_up']
    pt = pt.groupby((['scenario','input_draw'] + strata_cols)).sum()
    anemia_prev = anemia_pt / pt
    anemia_prev = anemia_prev.groupby((['scenario','measure'] + strata_cols)).mean() 
    return anemia_prev

In [23]:
overall = get_anemia_prevalence([])
overall

# we are seeing a VERY slight decrease in anemia prevalence between scenarios
# this is generally expected
# should see decrease in all anemia prevalences in iron scenario relative to baseline
    # an increase in anemia_none prevalence

value
scenario                               measure                  
baseline                               anemia_mild      0.251928
                                       anemia_moderate  0.385455
                                       anemia_none      0.326647
                                       anemia_severe    0.035970
folic_acid_fortification_scale_up      anemia_mild      0.251928
                                       anemia_moderate  0.385455
                                       anemia_none      0.326646
                                       anemia_severe    0.035971
iron_folic_acid_fortification_scale_up anemia_mild      0.251408
                                       anemia_moderate  0.381926
                                       anemia_none      0.331179
                                       anemia_severe    0.035488

In [24]:
by_year = get_anemia_prevalence(['year'])
by_year = by_year.reset_index().sort_values(by=['measure','year','scenario']).set_index(['measure','year','scenario'])
by_year

# this is displaying expected behavior
# no difference in 2020, slight differences in later years

value
measure         year scenario                                        
anemia_mild     2020 baseline                                0.251924
                     folic_acid_fortification_scale_up       0.251924
                     iron_folic_acid_fortification_scale_up  0.251924
                2021 baseline                                0.251958
                     folic_acid_fortification_scale_up       0.251958
                     iron_folic_acid_fortification_scale_up  0.251296
                2022 baseline                                0.251873
                     folic_acid_fortification_scale_up       0.251873
                     iron_folic_acid_fortification_scale_up  0.251141
                2023 baseline                                0.251958
                     folic_acid_fortification_scale_up       0.251959
                     iron_folic_acid_fortification_scale_up  0.251285
anemia_moderate 2020 baseline                                0.386428
                     folic_acid_fortification_scale_up       0.386428
                     iron_folic_acid_fortification_scale_up  0.386428
                2021 baseline                                0.385415
                     folic_acid_fortification_scale_up       0.385415
                     iron_folic_acid_fortification_scale_up  0.381120
                2022 baseline                                0.385168
                     folic_acid_fortification_scale_up       0.385168
                     iron_folic_acid_fortification_scale_up  0.380241
                2023 baseline                                0.384838
                     folic_acid_fortification_scale_up       0.384838
                     iron_folic_acid_fortification_scale_up  0.380039
anemia_none     2020 baseline                                0.325506
                     folic_acid_fortification_scale_up       0.325506
                     iron_folic_acid_fortification_scale_up  0.325506
                2021 baseline                                0.326619
                     folic_acid_fortification_scale_up       0.326619
                     iron_folic_acid_fortification_scale_up  0.332157
                2022 baseline                                0.327053
                     folic_acid_fortification_scale_up       0.327052
                     iron_folic_acid_fortification_scale_up  0.333389
                2023 baseline                                0.327374
                     folic_acid_fortification_scale_up       0.327372
                     iron_folic_acid_fortification_scale_up  0.333506
anemia_severe   2020 baseline                                0.036142
                     folic_acid_fortification_scale_up       0.036142
                     iron_folic_acid_fortification_scale_up  0.036142
                2021 baseline                                0.036008
                     folic_acid_fortification_scale_up       0.036008
                     iron_folic_acid_fortification_scale_up  0.035427
                2022 baseline                                0.035906
                     folic_acid_fortification_scale_up       0.035906
                     iron_folic_acid_fortification_scale_up  0.035229
                2023 baseline                                0.035830
                     folic_acid_fortification_scale_up       0.035831
                     iron_folic_acid_fortification_scale_up  0.035171

In [25]:
by_age = get_anemia_prevalence(['age_group'])
by_age = by_age.reset_index().sort_values(by=['measure','age_group','scenario']).set_index(['measure','age_group','scenario'])
by_age

# this is showing expected behavior... no change in early/late neonatal age groups, but slight change in others

value
measure         age_group      scenario                                        
anemia_mild     1_to_4         baseline                                0.255715
                               folic_acid_fortification_scale_up       0.255715
                               iron_folic_acid_fortification_scale_up  0.255035
                early_neonatal baseline                                0.096747
                               folic_acid_fortification_scale_up       0.096747
                               iron_folic_acid_fortification_scale_up  0.096747
                late_neonatal  baseline                                0.057254
                               folic_acid_fortification_scale_up       0.057254
                               iron_folic_acid_fortification_scale_up  0.057254
                post_neonatal  baseline                                0.252241
                               folic_acid_fortification_scale_up       0.252241
                               iron_folic_acid_fortification_scale_up  0.252326
anemia_moderate 1_to_4         baseline                                0.360188
                               folic_acid_fortification_scale_up       0.360188
                               iron_folic_acid_fortification_scale_up  0.355832
                early_neonatal baseline                                0.789831
                               folic_acid_fortification_scale_up       0.789831
                               iron_folic_acid_fortification_scale_up  0.789832
                late_neonatal  baseline                                0.798547
                               folic_acid_fortification_scale_up       0.798548
                               iron_folic_acid_fortification_scale_up  0.798548
                post_neonatal  baseline                                0.453166
                               folic_acid_fortification_scale_up       0.453166
                               iron_folic_acid_fortification_scale_up  0.452695
anemia_none     1_to_4         baseline                                0.353017
                               folic_acid_fortification_scale_up       0.353017
                               iron_folic_acid_fortification_scale_up  0.358641
                early_neonatal baseline                                0.003527
                               folic_acid_fortification_scale_up       0.003527
                               iron_folic_acid_fortification_scale_up  0.003527
                late_neonatal  baseline                                0.001004
                               folic_acid_fortification_scale_up       0.001004
                               iron_folic_acid_fortification_scale_up  0.001004
                post_neonatal  baseline                                0.247152
                               folic_acid_fortification_scale_up       0.247151
                               iron_folic_acid_fortification_scale_up  0.247634
anemia_severe   1_to_4         baseline                                0.031080
                               folic_acid_fortification_scale_up       0.031080
                               iron_folic_acid_fortification_scale_up  0.030492
                early_neonatal baseline                                0.109895
                               folic_acid_fortification_scale_up       0.109895
                               iron_folic_acid_fortification_scale_up  0.109895
                late_neonatal  baseline                                0.143195
                               folic_acid_fortification_scale_up       0.143195
                               iron_folic_acid_fortification_scale_up  0.143195
                post_neonatal  baseline                                0.047441
                               folic_acid_fortification_scale_up       0.047442
                               iron_folic_acid_fortification_scale_up  0.047345

## Conclusions

- Nothing unexpected here!

# 5. VAD and affected causes

In [26]:
deaths = pd.read_hdf(output_dirs[0] + 'deaths.hdf')
deaths.head()

,year,age_group,cause,folic_acid_fortification_group,vitamin_a_fortification_group,measure,input_draw,scenario,value
0,2020,1_to_4,diarrheal_diseases,covered,covered,death,21,baseline,0.0
1,2020,1_to_4,diarrheal_diseases,covered,covered,death,21,folic_acid_fortification_scale_up,0.0
2,2020,1_to_4,diarrheal_diseases,covered,covered,death,21,iron_folic_acid_fortification_scale_up,0.0
3,2020,1_to_4,diarrheal_diseases,covered,covered,death,21,vitamin_a_fortification_scale_up,0.0
4,2020,1_to_4,diarrheal_diseases,covered,covered,death,29,baseline,0.0


In [27]:
np.unique(deaths['cause'])

array(['diarrheal_diseases', 'lower_respiratory_infections', 'measles',
       'neural_tube_defects', 'other_causes'], dtype=object)

In [28]:
# vad should affect LRI, measles, DD, but NOT NTDs

In [29]:
def get_csmrs(strata_cols):
    deaths = pd.read_hdf(output_dirs[0] + 'deaths.hdf')
    pt = pd.read_hdf(output_dirs[0] + 'state_person_time.hdf')
    pt = pt.loc[pt.cause.str.contains('diarrheal')]
    deaths = deaths.groupby((['scenario','input_draw'] + strata_cols + ['cause'])).sum()
    pt = pt.groupby((['scenario','input_draw'] + strata_cols)).sum()
    csmrs = deaths / pt
    csmrs = csmrs.reset_index()
    csmrs = csmrs.loc[csmrs.scenario != 'iron_fortification_scale_up']
    csmrs = csmrs.groupby((['cause'] + strata_cols + ['scenario'])).mean()
    return csmrs

In [30]:
overall = get_csmrs([])
overall

# looks good!
# CSMR for other causes may be slightly higher in vitamin A scenario because 
# simulants who did not die of affected causes can now die of another cause

input_draw  \
cause                        scenario                                             
diarrheal_diseases           baseline                                    437.52   
                             folic_acid_fortification_scale_up           437.52   
                             iron_folic_acid_fortification_scale_up      437.52   
                             vitamin_a_fortification_scale_up            437.52   
lower_respiratory_infections baseline                                    437.52   
                             folic_acid_fortification_scale_up           437.52   
                             iron_folic_acid_fortification_scale_up      437.52   
                             vitamin_a_fortification_scale_up            437.52   
measles                      baseline                                    437.52   
                             folic_acid_fortification_scale_up           437.52   
                             iron_folic_acid_fortification_scale_up      437.52   
                             vitamin_a_fortification_scale_up            437.52   
neural_tube_defects          baseline                                    437.52   
                             folic_acid_fortification_scale_up           437.52   
                             iron_folic_acid_fortification_scale_up      437.52   
                             vitamin_a_fortification_scale_up            437.52   
other_causes                 baseline                                    437.52   
                             folic_acid_fortification_scale_up           437.52   
                             iron_folic_acid_fortification_scale_up      437.52   
                             vitamin_a_fortification_scale_up            437.52   

                                                                        value  
cause                        scenario                                          
diarrheal_diseases           baseline                                0.003170  
                             folic_acid_fortification_scale_up       0.003170  
                             iron_folic_acid_fortification_scale_up  0.003169  
                             vitamin_a_fortification_scale_up        0.003123  
lower_respiratory_infections baseline                                0.003606  
                             folic_acid_fortification_scale_up       0.003606  
                             iron_folic_acid_fortification_scale_up  0.003605  
                             vitamin_a_fortification_scale_up        0.003592  
measles                      baseline                                0.000517  
                             folic_acid_fortification_scale_up       0.000517  
                             iron_folic_acid_fortification_scale_up  0.000517  
                             vitamin_a_fortification_scale_up        0.000506  
neural_tube_defects          baseline                                0.000223  
                             folic_acid_fortification_scale_up       0.000218  
                             iron_folic_acid_fortification_scale_up  0.000218  
                             vitamin_a_fortification_scale_up        0.000223  
other_causes                 baseline                                0.013613  
                             folic_acid_fortification_scale_up       0.013613  
                             iron_folic_acid_fortification_scale_up  0.013610  
                             vitamin_a_fortification_scale_up        0.013613

In [31]:
by_age = get_csmrs(['age_group'])
by_age

# this is behaving as expected
# lower CSMRs for postneonatal and 1-4 age group, but not neonatal age groups

input_draw  \
cause                        age_group      scenario                                             
diarrheal_diseases           1_to_4         baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
                                            iron_folic_acid_fortification_scale_up      437.52   
                                            vitamin_a_fortification_scale_up            437.52   
                             early_neonatal baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
                                            iron_folic_acid_fortification_scale_up      437.52   
                                            vitamin_a_fortification_scale_up            437.52   
                             late_neonatal  baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
                                            iron_folic_acid_fortification_scale_up      437.52   
                                            vitamin_a_fortification_scale_up            437.52   
                             post_neonatal  baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
                                            iron_folic_acid_fortification_scale_up      437.52   
                                            vitamin_a_fortification_scale_up            437.52   
lower_respiratory_infections 1_to_4         baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
                                            iron_folic_acid_fortification_scale_up      437.52   
                                            vitamin_a_fortification_scale_up            437.52   
                             early_neonatal baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
                                            iron_folic_acid_fortification_scale_up      437.52   
                                            vitamin_a_fortification_scale_up            437.52   
                             late_neonatal  baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
                                            iron_folic_acid_fortification_scale_up      437.52   
                                            vitamin_a_fortification_scale_up            437.52   
                             post_neonatal  baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
...                                                                                        ...   
neural_tube_defects          1_to_4         iron_folic_acid_fortification_scale_up      437.52   
                                            vitamin_a_fortification_scale_up            437.52   
                             early_neonatal baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
                                            iron_folic_acid_fortification_scale_up      437.52   
                                            vitamin_a_fortification_scale_up            437.52   
                             late_neonatal  baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
                                            iron_folic_acid_fortif

In [32]:
by_year = get_csmrs(['year'])
by_year

# this is behaving as we would expect with the CORRECT population coverage algorithm (no change until 2020)

input_draw  \
cause                        year scenario                                             
diarrheal_diseases           2020 baseline                                    437.52   
                                  folic_acid_fortification_scale_up           437.52   
                                  iron_folic_acid_fortification_scale_up      437.52   
                                  vitamin_a_fortification_scale_up            437.52   
                             2021 baseline                                    437.52   
                                  folic_acid_fortification_scale_up           437.52   
                                  iron_folic_acid_fortification_scale_up      437.52   
                                  vitamin_a_fortification_scale_up            437.52   
                             2022 baseline                                    437.52   
                                  folic_acid_fortification_scale_up           437.52   
                                  iron_folic_acid_fortification_scale_up      437.52   
                                  vitamin_a_fortification_scale_up            437.52   
                             2023 baseline                                    437.52   
                                  folic_acid_fortification_scale_up           437.52   
                                  iron_folic_acid_fortification_scale_up      437.52   
                                  vitamin_a_fortification_scale_up            437.52   
lower_respiratory_infections 2020 baseline                                    437.52   
                                  folic_acid_fortification_scale_up           437.52   
                                  iron_folic_acid_fortification_scale_up      437.52   
                                  vitamin_a_fortification_scale_up            437.52   
                             2021 baseline                                    437.52   
                                  folic_acid_fortification_scale_up           437.52   
                                  iron_folic_acid_fortification_scale_up      437.52   
                                  vitamin_a_fortification_scale_up            437.52   
                             2022 baseline                                    437.52   
                                  folic_acid_fortification_scale_up           437.52   
                                  iron_folic_acid_fortification_scale_up      437.52   
                                  vitamin_a_fortification_scale_up            437.52   
                             2023 baseline                                    437.52   
                                  folic_acid_fortification_scale_up           437.52   
...                                                                              ...   
neural_tube_defects          2020 iron_folic_acid_fortification_scale_up      437.52   
                                  vitamin_a_fortification_scale_up            437.52   
                             2021 baseline                                    437.52   
                                  folic_acid_fortification_scale_up           437.52   
                                  iron_folic_acid_fortification_scale_up      437.52   
                                  vitamin_a_fortification_scale_up            437.52   
                             2022 baseline                                    437.52   
                                  folic_acid_fortification_scale_up           437.52   
                                  iron_folic_acid_fortification_scale_up      437.52   
                                  vitamin_a_fortification_scale_up            437.52   
                             2023 baseline                                    437.52   
                                  folic_acid_fortification_scale_up           437.52   
                                  iron_folic_acid_fortification_scale_up      437.52   
                          

## Conclusions

This is looking as expected (VAD prevalence appears to be affecting CSMRs due to diarrheal diseases, measles, and LRI.

However, it also appears to be affecting "other causes" which is unexpected